In [214]:
import os
import pandas as pd
from os import walk
import psycopg2
from psycopg2 import sql
from psycopg2.sql import Identifier, SQL
from datetime import datetime
import numpy as np
from tabulate import tabulate
from sqlalchemy import create_engine
import re
import psycopg2.extras

connection_string = "user='nyc_covid' password='nyc_covid' dbname='nyc_covid' "
conn = psycopg2.connect(connection_string)
def __init__(connection_string):
        conn = psycopg2.connect(connection_string)    


In [191]:
def query1(self):
    '''
         Hospital Information according to Zip code, with Google Maps location
         Input: Zip code (INT)
         Output: Pandas Dataframe 
    '''
    try:
        zip_code = int(input("Enter a 5 digit Zip Code to retrive Hospital Information==>"))
        
    except:
        print("Incorrect ZIP code")
        
    try:
        cursor = conn.cursor()
#         print('test')

        query = ('''SELECT h.objectid, h.id, h.name, a.address, a.city, a.county, a.state, a.zip, a.latitude, a.longitude, h.type, c.telephone, c.website \
        FROM hospital_data h JOIN address_data a ON h.objectid = a.objectid \
        AND h.id = a.id JOIN contact_data c ON h.objectid = c.objectid AND h.id = c.id \
        WHERE a.zip = {c1};''').format(c1=zip_code)
        cursor.execute(query)
        records = cursor.fetchall()
        headers = ['ObjectID', 'ID', 'Name', 'Address', 'City', 'County', 'State', \
                    'ZIP', 'Latitude','Longitude', 'Type', 'Telephone', 'Website' ]
#         print(records)
        table = []
        for i in records:
            
            
            table.append([i[0], i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10],i[11],i[12]])
        df = pd.DataFrame(table, columns = headers)  
#         print(table)
#         table = [[records[i][0], records[i][1], records[i][2], records[i][3], records[i][4], records[i][5],records[i][6],records[i][7],records[i][8],records[i][9],records[i][10],records[i][11], records[i][12],records[i][13] ]]
#         print(tabulate(table, headers))  
#         print(df)
        return df
    except:
        print("Something went wrong2")
            

In [173]:
query1(_)

Enter a 5 digit Zip Code to retrive Hospital Information==>12180


,ObjectID,ID,Name,Address,City,County,State,ZIP,Latitude,Longitude,Type,Telephone,Website
0,275,9912180,SAMARITAN HOSPITAL,2215 BURDETT AVE,TROY,RENSSELAER,NY,12180,42.7333,-73.6722,GENERAL ACUTE CARE,(518) 271-3300,http://www.nehealth.com
1,353,117712180,THE BURDETT CARE CENTER,2215 BURDETT AVENUE,TROY,RENSSELAER,NY,12180,42.7334,-73.6727,WOMEN,(518) 271-3655,http://www.burdettcarecenter.org/
2,6146,9812180,SAMARITAN HOSPITAL ST MARY'S,1300 MASSACHUSETTS AVE,TROY,RENSSELAER,NY,12180,42.7427,-73.6764,GENERAL ACUTE CARE,(518) 268-5000,http://www.setonhealth.org/st_marys/index.cfm


In [174]:
def query2():
    '''
         Giving all information avalable for Covid cases in a particular county and date range
         Input: county,start date, end date
         Output: Pandas Dataframe 
    '''
    try:
        county = str(input('Input county to check for all information avalable for Covid cases from Jan to Oct 2021 ==> ')).lower().strip()
        year = 2021
        s_month = int(input('Input a START month between 1-10 ==> ').strip())
        s_day = int(input('Input a START date between 1-31 ==> ').strip())
        e_month = int(input('Input a END month between 1-10 ==> ').strip())
        e_day = int(input('Input a END date between 1-31 ==> ').strip())
        m_31 = [1,3,5,7,8,10]
        m_30 = [2,4,6,9]
        if s_month in m_31:
            if 0< s_day <= 31:
                s_date = str(year) + '-' + str(s_month) + '-' + str(s_day)
            elif 0< s_day <= 31:
                s_date = str(year) + '-' + str(s_month) + '-' + str(s_day)
            else:
                raise Exception('DATE out of range')
        elif s_month in m_30:
            if 0 < s_day <= 30:
                s_date = str(year) + '-' + str(s_month) + '-' + str(s_day)
            elif 0 < s_day <= 30:
                s_date = str(year) + '-' + str(s_month) + '-' + str(s_day)
            else:
                raise Exception('DATE out of range')
        if e_month in m_31:
            if 0< e_day <= 31:
                e_date = str(year) + '-' + str(e_month) + '-' + str(e_day)
            elif 0< e_day <= 31:
                e_date = str(year) + '-' + str(e_month) + '-' + str(e_day)
            else:
                raise Exception('DATE out of range')
        elif e_month in m_30:
            if 0 < e_day <= 30:
                e_date = str(year) + '-' + str(e_month) + '-' + str(e_day)
            elif 0 < e_day <= 30:
                e_date = str(year) + '-' + str(e_month) + '-' + str(e_day)
            else:
                raise Exception('DATE out of range')
        else:
            raise Exception('MONTH out of range')
    
    except :
        print("Incorrect Date(s)")
        
        
    
    try:
        cursor = conn.cursor()
#         print('test')

        query = ('''SELECT date, state, cases, cases_avg, cases_avg_per_100k, deaths, deaths_avg, deaths_avg_per_100k \
                    FROM counties WHERE LOWER(COUNTY) = %s AND DATE BETWEEN %s AND %s;''')
#         print('till here fine')
        cursor.execute(query, (str(county), str(s_date), str(e_date)))
        

        records = cursor.fetchall()
#         print(records)
        headers = ['Date', 'State', 'Cases', 'Cases_Avg', 'Cases_Avg_per_100k ', 'Deaths', 'Deaths_Avg','Deaths_Avg_per_100k']
#         print(records)
        table = []
        
        for i in records:
            
            table.append([i[0], i[1],i[2],i[3],i[4],i[5],i[6],i[7]])
#         print(table)
        df = pd.DataFrame(table) 
        df.columns = headers
        return df
    except Exception as error:
        print ("Oops! An exception has occured:", error)
        print ("Exception TYPE:", type(error))


In [176]:
query2()



In [228]:
def query3():
    '''
         First show the NAICS Code and its description then make the user choose the Naics code
         Giving a list of all hospitals with address for a particular NAICS code and ZIP code
         Input: NAICS CODE,  ZIP CODE
         Output: Pandas Dataframe 
    '''
    cursor = conn.cursor()
    query =('''SELECT DISTINCT(NAICS_CODE), NAICS_DESC FROM hospital_data;''')
    cursor.execute(query)
    records = cursor.fetchall()
    headers = ['NAICS Codes', 'NAICS Description']
    table = []
    for i in records:
        table.append([i[0], i[1]])
    df = pd.DataFrame(table, columns = headers) 
    print(tabulate(table, headers))  
    print('''
            Enter a NAICS Code from the above table to view all hospitals 
            with address for a that particular NAICS code and ZIP code.
            ''')
    try:
        Naics_code = int(input('Enter a NAICS Code from the above table ==> ').strip())
        zip_code = int(input('Enter a zip code to view all hospitals ==> ').strip())
#         print(df['NAICS Codes'])
        if Naics_code not in list(df['NAICS Codes']):
            raise Exception('NAICS Code not in above list')
        query =('''SELECT h.objectid, h.id, h.name, h.type, h.beds, a.address, a.city, a.county, a.state, a.zip, a.latitude, a.longitude \
                    FROM hospital_data h JOIN address_data a ON h.objectid = a.objectid AND h.id = a.id \
                    WHERE a.zip = {};''').format(zip_code)
        cursor.execute(query)
        records = cursor.fetchall()
        headers = ['ObjectID', 'ID', 'Name','Type', 'Address', 'City', 'County', 'State', \
                    'ZIP', 'Latitude','Longitude']
        table = []
        for i in records:
            table.append([i[0], i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10]])
        df = pd.DataFrame(table, columns = headers)
        return df
    except Exception as error:
        print ("Oops! An exception has occured:", error)
        print ("Exception TYPE:", type(error))
#     try:
#         NAICS_Code = int(input(''))

In [229]:
query3()

  NAICS Codes  NAICS Description
-------------  ---------------------------------------------------------------------
       622310  CHRONIC DISEASE HOSPITALS
       622310  ORTHOPEDIC HOSPITALS
       622110  GENERAL MEDICAL AND SURGICAL HOSPITALS
       622210  CHILDREN'S HOSPITALS, PSYCHIATRIC OR SUBSTANCE ABUSE
       622310  EXTENDED CARE HOSPITALS (EXCEPT MENTAL, SUBSTANCE ABUSE)
       622210  HOSPITALS, ADDICTION
       622210  HOSPITALS, PSYCHIATRIC (EXCEPT CONVALESCENT)
       622210  HOSPITALS, PSYCHIATRIC PEDIATRIC
       622310  HOSPITALS, SPECIALTY (EXCEPT PSYCHIATRIC, SUBSTANCE ABUSE)
       622310  SPECIALTY (EXCEPT PSYCHIATRIC AND SUBSTANCE ABUSE) HOSPITALS
       622210  PSYCHIATRIC AND SUBSTANCE ABUSE HOSPITALS
       622310  CHILDREN'S HOSPITALS, SPECIALTY (EXCEPT PSYCHIATRIC, SUBSTANCE ABUSE)
       622310  REHABILITATION HOSPITALS (EXCEPT ALCOHOLISM, DRUG ADDICTION)
       622310  MATERNITY HOSPITALS
       622210  HOSPITALS, SUBSTANCE ABUSE
       622110  CHILDRE

,ObjectID,ID,Name,Type,Address,City,County,State,ZIP,Latitude,Longitude
0,4118,4690015,CALIFORNIA HOSPITAL MEDICAL CENTER - LOS ANGELES,GENERAL ACUTE CARE,318,1401 SOUTH GRAND AVENUE,LOS ANGELES,LOS ANGELES,CA,90015,34.0374


In [235]:
def query4():
    '''
         The date a hospital was last reviewed in a particular zip code
         Input: ZIP CODE
         Output: Pandas Dataframe 
    '''
    try:
        zip_code = int(input('Enter a zip code to view all hospitals ==> ').strip())
        query =('''SELECT h.objectid, h.id, h.name, h.status, h.type, h.trauma, h.owner, h.ttl_staff, h.population, h.beds, h.helipad, m.source, m.sourcedate, m.val_method, m.val_date, a.zip \
                    FROM hospital_data h JOIN metadata m ON h.objectid = m.objectid \
                    AND h.id = m.id JOIN address_data a ON h.objectid = a.objectid \
                    AND h.id = a.id WHERE a.zip = {};''').format(zip_code)
        cursor.execute(query)
        records = cursor.fetchall()
        headers = ['ObjectID', 'ID', 'Name','Status','Type', 'Trauma', 'Owner', 'TTL_staff', 'Population','Beds', 'Helipad', 'Source', 'Source Date', 'Val Method','Val Date',  \
                    'ZIP']
        table = []
        for i in records:
            table.append([i[0], i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10],i[11],i[12],i[13],i[14],i[15],])
        df = pd.DataFrame(table, columns = headers)
        return df
    except Exception as error:
        print ("Oops! An exception has occured:", error)
        print ("Exception TYPE:", type(error))


In [234]:
query4()

Enter a zip code to view all hospitals ==> 12180


,ObjectID,ID,Name,Status,Type,Trauma,Owner,TTL_staff,Population,Beds,Helipad,Source,Source Date,Val Method,Val Date,ZIP
0,275,9912180,SAMARITAN HOSPITAL,OPEN,GENERAL ACUTE CARE,NOT AVAILABLE,NON-PROFIT,-999,212,212,N,https://profiles.health.ny.gov/hospital/county...,2019-08-20,IMAGERY,2014-02-10,12180
1,353,117712180,THE BURDETT CARE CENTER,OPEN,WOMEN,NOT AVAILABLE,NON-PROFIT,-999,15,15,N,https://profiles.health.ny.gov/hospital/county...,2019-08-20,IMAGERY/OTHER,2015-02-11,12180
2,6146,9812180,SAMARITAN HOSPITAL ST MARY'S,OPEN,GENERAL ACUTE CARE,NOT AVAILABLE,NON-PROFIT,-999,196,196,N,https://profiles.health.ny.gov/hospital/county...,2019-08-20,IMAGERY,2014-02-10,12180


In [ ]:
z